In [1]:
import math
from pprint import pprint
from math import log, factorial, sqrt, comb
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
    permutations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter, xor, and_, or_, indexOf, contains
from bisect import bisect_right, bisect_left
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import sets
import matrix
import mathematics
import sequences
from importlib import reload

for m in (
    arrays,
    trees,
    lists,
    graphs,
    search,
    strings,
    sets,
    sequences,
    matrix,
    mathematics,
):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import sets
import matrix as mt
import mathematics as m
from sequences import find_if

print()

In [4]:
def serialize_in_order(n, lr: str = "()") -> str:
    "Return a DFS, in-order string representation using brackets."
    lb, rb = lr
    # Iterative implementation using a stack.
    s = []
    v = []
    while s or n:
        # Descend left from node.
        while n:
            s.append(n)
            s.append(n)
            n = n.left
        n = s.pop()
        if s and s[-1] == n:
            n = n.right
        else:
            lv = v.pop() if n.left else lr
            rv = v.pop() if n.right else lr
            if lv or rv:
                v.append(f"{lb}{lv} {str(n.data)} {rv}{rb}")
            else:
                v.append(str(n.data))
        print(f"{n.data if n else '_'}: {s}, {v}")
    return v[0]

In [3]:
t = tr.make("1 2 4 N 3")
t.display()
# print(f"ino: {serialize_in_order(t)}")

 _1 
/  \
2  4
 \  
 3  


In [148]:
def parse_arithmetic_expression(tokens: list) -> tuple:
    """Parses an arithmetic expression consisting of literals and operators from '+-*/()'.

    The parsed expression is represented as tripples of the form:
        (<operator>, <arg1>, <arg2>)

    Parameters:
        tokens (list): tokenized expression consisting of operators '+-*/()' and literals.

    Returns:
        tuple: the root of the parsed expression as (<operator>, <arg1>, <arg2>)
    """
    # The implementation here avoids recursion and uses two stacks
    # similar to the  Dijkstra's Shunting Yard algorithm.

    # Stores the arguments and successive parsed results of the expression.
    args = []
    # Stores the operators seen so far.
    # The ops arrays should contains "+-(" and at most one of "*/".
    # Note that we start with an implicit bracket.
    ops = ["("]

    def close_bracket():
        # This is called when we close and process a (bracketed) sub-expression
        #
        # The remaining operators here are "+-".
        # The idea is to first reverse the operators and their arguments.
        rargs = [args.pop()]
        rops = []
        while ops[-1] in "+-":
            rargs.append(args.pop())
            rops.append(ops.pop())
        # We should have arrived at the opening bracket.
        assert ops.pop() == "("
        # Apply the operators (in reversed order) to form the expressions.
        while rops:
            rargs.append((rops.pop(), rargs.pop(), rargs.pop()))
        # The final expression is moved back onto the stack.
        args.append(rargs.pop())

    for t in tokens:
        # Push all operators onto the `ops` stack
        # except for the closing bracket.
        if t in "*/+-(":
            ops.append(t)
            continue

        if t == ")":
            # Closing bracket forms on new expression on the args stack.
            close_bracket()
        else:
            # Push remaining args onto the stack.
            args.append(t)

        # Given a new expression on the stack from above,
        # we can now form any "*/" expressions if possible.
        # This removes all the "*/" expressions from the ops stack.
        while ops[-1] in "*/":
            b, a = args.pop(), args.pop()
            args.append((ops.pop(), a, b))

    # Finally, we close the first implicit bracket.
    close_bracket()
    # And return the remaining only expression on the args stack.
    assert len(args) == 1
    return args[0]

In [ ]:
def format_binary_operator_expression(e) -> str:
    "Format expressions consisting of (<operator>, <arg1>, <arg2>)."
    # This is a basic depth first algorithm.
    o = ""

    def dfs(c):
        s = []
        while s or c:
            while c:
                s.append(c)
                c = type(c) != str and c[1]
            c = s.pop()
            yield c
            c = type(c) != str and c[2]

    def p(s):
        if len(s) == 1:
            return s
        if e[0] == "+" or (e[0] in "*-" and s[0] in "*/"):
            return format(s)
        return "(" + format(s) + ")"

    return format(e[1]) + e[0] + p(e[2]) if len(e) == 3 else e


def format_binary_expression(op, a, b):
    def f(arg):
        return arg if type(arg) == str else arg[1]

    def p(arg):
        if type(arg) == str or op == "+" or op in "*-" and arg[0] in "*/":
            return f(arg)
        return "(" + f(arg) + ")"

    return op, f(a) + op + p(b)

In [149]:
parse_arithmetic_expression("a-b-c")

('-', ('-', 'a', 'b'), 'c')

In [136]:
parse("A-B-C")

A: ['A'], ['(']
B: ['A', 'B'], ['(', '-']
C: ['A', 'B', 'C'], ['(', '-', '-']
): [('-', 'A-B-C')], []


'A-B-C'

In [137]:
e = parse("A+B*C")
e

A: ['A'], ['(']
B: ['A', 'B'], ['(', '+']
C: ['A', ('*', 'B*C')], ['(', '+']
): [('+', 'A+B*C')], []


'A+B*C'

In [138]:
e = parse("(A/(B*C))")
e

A: ['A'], ['(', '(']
B: ['A', 'B'], ['(', '(', '/', '(']
C: ['A', ('*', 'B*C')], ['(', '(', '/', '(']
): ['A', ('*', 'B*C')], ['(', '(', '/']
): [('/', 'A/(B*C)')], ['(', '(']
): [('/', 'A/(B*C)')], ['(']
): [('/', 'A/(B*C)')], ['(']
): [('/', 'A/(B*C)')], []


'A/(B*C)'

In [139]:
e = parse("A/(B/C)")
e

A: ['A'], ['(']
B: ['A', 'B'], ['(', '/', '(']
C: ['A', ('/', 'B/C')], ['(', '/', '(']
): ['A', ('/', 'B/C')], ['(', '/']
): [('/', 'A/(B/C)')], ['(']
): [('/', 'A/(B/C)')], []


'A/(B/C)'

In [140]:
e = parse("X/Y/Z/(A+B)")
e

X: ['X'], ['(']
Y: [('/', 'X/Y')], ['(']
Z: [('/', 'X/Y/Z')], ['(']
A: [('/', 'X/Y/Z'), 'A'], ['(', '/', '(']
B: [('/', 'X/Y/Z'), 'A', 'B'], ['(', '/', '(', '+']
): [('/', 'X/Y/Z'), ('+', 'A+B')], ['(', '/']
): [('/', 'X/Y/Z/(A+B)')], ['(']
): [('/', 'X/Y/Z/(A+B)')], []


'X/Y/Z/(A+B)'

In [141]:
parse("X/Y/(Z/(A+B))")

X: ['X'], ['(']
Y: [('/', 'X/Y')], ['(']
Z: [('/', 'X/Y'), 'Z'], ['(', '/', '(']
A: [('/', 'X/Y'), 'Z', 'A'], ['(', '/', '(', '/', '(']
B: [('/', 'X/Y'), 'Z', 'A', 'B'], ['(', '/', '(', '/', '(', '+']
): [('/', 'X/Y'), 'Z', ('+', 'A+B')], ['(', '/', '(', '/']
): [('/', 'X/Y'), ('/', 'Z/(A+B)')], ['(', '/', '(']
): [('/', 'X/Y'), ('/', 'Z/(A+B)')], ['(', '/']
): [('/', 'X/Y/(Z/(A+B))')], ['(']
): [('/', 'X/Y/(Z/(A+B))')], []


'X/Y/(Z/(A+B))'

In [142]:
e = parse("A-(B-C)")
format(e)

A: ['A'], ['(']
B: ['A', 'B'], ['(', '-', '(']
C: ['A', 'B', 'C'], ['(', '-', '(', '-']
): ['A', ('-', 'B-C')], ['(', '-']
): ['A', ('-', 'B-C')], ['(', '-']
): [('-', 'A-(B-C)')], []


'A-(B-C)'

In [143]:
e = parse("A+(B-C)")
format(e)

A: ['A'], ['(']
B: ['A', 'B'], ['(', '+', '(']
C: ['A', 'B', 'C'], ['(', '+', '(', '-']
): ['A', ('-', 'B-C')], ['(', '+']
): ['A', ('-', 'B-C')], ['(', '+']
): [('+', 'A+B-C')], []


'A+B-C'

In [ ]:
# Solution from https://auth.geeksforgeeks.org/user/husoski/practice

import re


class Solution:
    # A very simple regex-based tokenizer:

    def lexscan(self, source):
        TOK_PAT = re.compile(
            "|".join(
                (
                    r"(?P<OP>[-+*/])",
                    r"(?P<IDENT>[A-Z]+)",
                    r"(?P<LPAR>\()",
                    r"(?P<RPAR>\))",
                    r"(?P<ERROR>.)",
                )
            )
        )

        for m in TOK_PAT.finditer(source):
            yield (m.lastgroup, m.group())

    # Convert infix expression to postfix ("reverse Polish" form), using
    # Dijkstra's "shunting yard" approach, with operator precedence:

    def to_postfix(self, source):
        precedence = {"+": 1, "-": 1, "*": 2, "/": 2, "(": 0}
        op_stack = []
        result = []
        for ttyp, tstr in self.lexscan(source):
            if ttyp == "IDENT":
                result.append(tstr)
            elif ttyp == "LPAR":
                op_stack.append("(")
            elif ttyp == "OP":
                prec = precedence[tstr]
                while op_stack and precedence[op_stack[-1]] >= prec:
                    result.append(op_stack.pop())
                op_stack.append(tstr)
            elif ttyp == "RPAR":
                while op_stack and op_stack[-1] != "(":
                    result.append(op_stack.pop())
                op_stack.pop()
            else:
                raise RuntimeError("bad token: " + tstr)

        while op_stack:
            op = op_stack.pop()
            assert op != "("
            result.append(op)

        return result

    # Convert postfix ("reverse Polish") directly to infix.  Stack based approach since
    # final testcase nesting is too deep for recursion using old Python 3.5 at GFG.
    # (Increasing recursion limit leads to stack crash):

    def postfix_to_infix(self, postfix):
        # simulate evaluation of the postfix expression
        stack = []
        for tok in postfix:
            if tok >= "A":
                stack.append([tok, ""])
            else:
                op = tok
                rhs, rhop = stack.pop()
                lhs, lhop = stack.pop()

                lhpar = rhpar = False  # default to no parentheses on lhs,rhs
                if op == "+":
                    if rhop == "-":
                        op = "-"
                elif op == "-":
                    if rhop in ("+", "-"):
                        rhpar = True
                elif op == "*":
                    if lhop in ("+", "-"):
                        lhpar = True
                    if rhop in ("+", "-"):
                        rhpar = True
                    elif rhop == "/":
                        op = rhop
                elif op == "/":
                    if rhop in ("+", "-", "*", "/"):
                        rhpar = True
                else:
                    0 / 0  ###TODO

                if lhpar:
                    lhs = lhs.join("()")
                if rhpar:
                    rhs = rhs.join("()")
                stack.append([lhs + tok + rhs, op])

        result, op = stack.pop()
        assert stack == []
        return result

    def removeBrackets(self, Exp):
        try:
            postfix = self.to_postfix(Exp)
            # print(*postfix)
            result = self.postfix_to_infix(postfix)
            return result

        except:
            import sys, traceback

            _, e, tb = sys.exc_info()
            print(e)
            traceback.print_tb(tb, limit=2, file=sys.stdout)